<a href="https://colab.research.google.com/github/muhajirakbarhsb/NLP_class_2023/blob/main/Week_Class_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers peft  accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 33.4 MB/s 

In [1]:
import pickle
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader


In [2]:
# import dependencies
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

import os
from google.colab import drive

import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [3]:
# specify model huggingface mode name
model_name = "anakin87/zephyr-7b-alpha-sharded"

###### other models:
# "Trelis/Llama-2-7b-chat-hf-sharded-bf16"
# "bn22/Mistral-7B-Instruct-v0.1-sharded"
# "HuggingFaceH4/zephyr-7b-beta"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [4]:
# fucntion for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [5]:
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
loader = CSVLoader(file_path='stat.csv', encoding="unicode_escape", csv_args={
                'delimiter': ','})
docs = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(docs)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #Chage the chunk_size and chunk_overlap as needed
# all_splits = text_splitter.split_documents(documents)

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

#embed document chunks
vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="chroma_db2")

# specify the retriever
retriever = vectordb.as_retriever()

In [7]:
docs

[Document(page_content='Capaian Pembelajaran Mata Kuliah (CP-MK): Mampu memahami konsep Ekonomi Mikro dan Ekonomi Makro dengan menggunakan pendekatan verbal, pendekatan grafik serta pendekatan matematis.\nBidang: Bidang Statistika Bisnis dan Industri', metadata={'source': 'stat.csv', 'row': 0}),
 Document(page_content='Capaian Pembelajaran Mata Kuliah (CP-MK): Mampu menguasai formulasi model program linear dan memahami serta dapat menghitung yang terkait dengan Metode Grafik, Metode Simpleks, Model Program Linear Ireguler, Analisis Post Optimal, dan Model Transportasi dan Penugasan.\nBidang: Bidang Statistika Bisnis dan Industri', metadata={'source': 'stat.csv', 'row': 1}),
 Document(page_content='Capaian Pembelajaran Mata Kuliah (CP-MK): Mampu menerapkan metode statistika dalam manajemen operasi.\nBidang: Bidang Statistika Bisnis dan Industri', metadata={'source': 'stat.csv', 'row': 2}),
 Document(page_content='Capaian Pembelajaran Mata Kuliah (CP-MK): Mampu menerapkan teori resiko di

In [8]:
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=700,
        do_sample=True,
        top_k=2,
        temperature=0.3,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)


In [9]:
# Build chain

template = """
Selalu ramah dan persuasif. Gunakan emoticon bila diperlukan.
Berikan jawaban yang komprehensif dan padat.
Jangan berikan jawaban jika keyword pada pertanyaan di luar konteks statistika unsyiah.
{context}
Question: {question}
Helpful Answer:
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template,)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.5, 'k': 2}),
    chain_type_kwargs = {
        "prompt": QA_CHAIN_PROMPT,
        }
    )

In [16]:
def chat(input):
    with get_openai_callback() as cb:
        result = qa_chain(input)
    print(result['result'])
    print(f'\nPrompt Tokens: {cb.prompt_tokens}')
    return result

In [17]:
%%time
a =chat("jelaskan tentang Bidang Statistika Bisnis dan Industri ")

Bidang Statistika Bisnis dan Industri merupakan bidang statistika yang mengkaji aspek-aspek statistik yang terkait dengan bisnis dan industri. Bidang ini meliputi pengembangan metode statistik untuk menghasilkan informasi yang diperlukan oleh pihak-pihak di dalam industri keuangan, seperti perusahaan, bank, dan lain-lain. 

Metode statistik yang digunakan di bidang Statistika Bisnis dan Industri antara lain:
1. Analisis data kualitatif
2. Analisis data kualitatif kuantitatif
3. Analisis data kuantitatif
4. Analisis data time series
5. Analisis data spasi waktu
6. Analisis data cluster
7. Analisis data factor
8. Analisis data regresi
9. Analisis data regresi menggunakan metode panel
10. Analisis data regresi menggunakan metode bayesian

Hasil analisis statistik yang diperoleh dapat digunakan untuk menentukan strategi bisnis, meningkatkan kualitas produk, mengurangi kemarahan, mengurangi biaya, dan meningkatkan keuntungan.

Capaian Pembelajaran Mata Kuliah (CP-MK): Mampu menerapkan teori

In [22]:
print(a['result'])

Bidang Statistika Bisnis dan Industri merupakan bidang statistika yang mengkaji aspek-aspek statistik yang terkait dengan bisnis dan industri. Bidang ini meliputi pengembangan metode statistik untuk menghasilkan informasi yang diperlukan oleh pihak-pihak di dalam industri keuangan, seperti perusahaan, bank, dan lain-lain. 

Metode statistik yang digunakan di bidang Statistika Bisnis dan Industri antara lain:
1. Analisis data kualitatif
2. Analisis data kualitatif kuantitatif
3. Analisis data kuantitatif
4. Analisis data time series
5. Analisis data spasi waktu
6. Analisis data cluster
7. Analisis data factor
8. Analisis data regresi
9. Analisis data regresi menggunakan metode panel
10. Analisis data regresi menggunakan metode bayesian

Hasil analisis statistik yang diperoleh dapat digunakan untuk menentukan strategi bisnis, meningkatkan kualitas produk, mengurangi kemarahan, mengurangi biaya, dan meningkatkan keuntungan.

Capaian Pembelajaran Mata Kuliah (CP-MK): Mampu menerapkan teori

In [23]:
%%time
b =chat("saya tidak bisa login")

/usr/local/lib/python3.10/dist-packages/langchain/schema/vectorstore.py:325: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


Pertanyaan: Saya tidak bisa login.

Penyebab:
1. Anda tidak memiliki akun.
2. Anda memasukkan kata sandi yang salah.
3. Anda memasukkan kata sandi yang tidak valid.
4. Anda memasukkan kata sandi yang tidak sesuai dengan format.
5. Anda memasukkan kata sandi yang tidak sesuai dengan ketentuan.

Solusi:
1. Jika anda tidak memiliki akun, silakan daftar terlebih dahulu.
2. Jika anda memasukkan kata sandi yang salah, silakan cek kata sandi anda. Jika anda tidak ingin mengganti kata sandi, silakan klik "Lupa Kata Sandi" di halaman login.
3. Jika anda memasukkan kata sandi yang tidak valid, silakan cek kata sandi anda. Jika anda tidak ingin mengganti kata sandi, silakan klik "Lupa Kata Sandi" di halaman login.
4. Jika anda memasukkan kata sandi yang tidak sesuai dengan format, silakan cek kata sandi anda. Jika anda tidak ingin mengganti kata sandi, silakan klik "Lupa Kata Sandi" di halaman login.
5. Jika anda memasukkan kata sandi yang tidak sesuai dengan ketentuan, silakan cek kata sandi and